Importar librerías

In [1]:
# Manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Leer y escribir fácilmente ficheros en formato CSV
import pandas as pd

# Libreria para gráficos
from matplotlib import pyplot

# Módulo de optimizacion en scipy
from scipy import optimize

# Le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Libreria pandas para leer el dataset.
datos = pd.read_csv('/content/gdrive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL/datasets/poker-hand-testing.csv')

In [4]:
datos

,Suit of Card 1,Rank of Card 1,Suit of Card 2,Rank of Card 2,Suit of Card 3,Rank of Card 3,Suit of Card 4,Rank of Card 4,Suit of Card 5,Rank of Card 5,Poker Hand
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,3,1,1,12,2,9,4,9,2,6,1
999996,3,3,4,5,2,7,1,4,4,3,1
999997,1,11,4,7,3,9,1,13,2,7,1
999998,3,11,1,8,1,1,3,13,2,8,1


Datos de clasificacion en números

In [5]:
datos['Poker Hand']. value_counts()

0    501209
1    422498
2     47622
3     21121
4      3885
5      1996
6      1424
7       230
8        12
9         3
Name: Poker Hand, dtype: int64

In [6]:
# La entrada es de 14 elemento contando con x0
input_layer_size  = 11

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 10

X = datos.drop(['Poker Hand'], axis=1).values

y = datos['Poker Hand'].values

m = y.size

In [7]:
print(X[0,:])
print(y)

[ 1  1  1 13  2  4  2  3  1 12]
[0 1 1 ... 1 1 2]


In [8]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [9]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [10]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape

X = X_norm

In [11]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

Cálculo de la sigmoide de z

In [12]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [13]:
def lrCostFunction(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

Clasificación One-vs-all

In [21]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [22]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 11)


In [23]:
print(all_theta)

[[ 4.84221228e-03  1.11988147e-03 -5.46366056e-03  1.77418289e-03
  -1.31332614e-03 -8.38304138e-04 -3.20777573e-03  2.24926455e-03
  -3.42323010e-03  2.10941898e-03  3.25273709e-04]
 [-3.12533596e-01 -1.60481345e-03  3.59709758e-03 -2.40560453e-03
   4.05196711e-04  5.40505024e-04  1.53630334e-03 -1.16843780e-03
   1.00046670e-03 -8.55696574e-04 -2.65720352e-03]
 [-2.99572986e+00 -7.19523905e-04 -5.75721005e-03  6.53433032e-04
  -5.82585439e-03 -2.93525708e-03 -1.93224486e-03 -3.70150984e-03
   2.69794228e-03 -2.28119401e-03  1.41286382e-04]
 [-3.83593510e+00  4.42009486e-03  1.57357411e-02  7.84694062e-03
   6.05500442e-03  2.56128292e-03  8.47605355e-03 -3.12676349e-03
   5.00284395e-03 -5.68060978e-03  9.45754395e-03]
 [-5.56450931e+00  1.15605665e-02  8.81756590e-02 -2.44212394e-02
   7.55249564e-02  1.05896669e-02  8.38687370e-02 -2.54389503e-02
   8.29196061e-02 -2.64470900e-02  9.78276353e-02]
 [-6.22069078e+00  2.27330074e-02  1.93168886e-02  2.33051232e-02
   2.48844657e-02  

In [24]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [25]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[10:150])

(1000000, 10)
Precision del conjuto de entrenamiento: 50.12%
(140, 10)
(140, 11)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 2 0 1 0 3 0 0 0 1 0 1 1 0 0 1 0
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 2 1 3 0 1 0 1 2 0 1 1 0 1 1 0 1 0 0 1 1
 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 2 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0
 2 1 1 0 1 0 1 1 1 1 1 5 1 0 0 0 0 1 0 0 0 4 2 1 1 0 0 0 1]
